# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [37]:
%pip install numpy pandas scikit-learn


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

# 1) Load data
# Expecting TSV with columns: month, price
df = pd.read_csv("strawberry-prices.tsv", sep="\t")

df.columns = [c.strip().lower() for c in df.columns]
if "month" not in df.columns or "price" not in df.columns:
    raise ValueError("Expected columns: 'month' and 'price' in strawberry-prices.tsv")

df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

# Keep 2020-2024 only (per instructions)
df = df[(df["month"] >= "2020-01-01") & (df["month"] <= "2024-12-31")].copy()

# Ensure a clean monthly index (Month Start)
df = df.set_index("month").asfreq("MS")

# If any missing prices/months exist, interpolate so features can be computed
if df["price"].isna().any():
    df["price"] = df["price"].interpolate(limit_direction="both")

# 2) Feature engineering
def make_features(series: pd.Series) -> pd.DataFrame:
    """Create features aligned to the series index."""
    X = pd.DataFrame(index=series.index)

    # Trend
    X["t"] = np.arange(len(series))

    # Month-of-year
    month_dummies = pd.get_dummies(series.index.month, prefix="m", drop_first=False)
    month_dummies.index = series.index
    X = pd.concat([X, month_dummies], axis=1)

    # Lags
    X["lag1"] = series.shift(1)
    X["lag12"] = series.shift(12)

    # Rolling means (shifted so they use only past data)
    X["roll3"] = series.shift(1).rolling(3).mean()
    X["roll6"] = series.shift(1).rolling(6).mean()

    return X

y_all = df["price"].copy()
X_all = make_features(y_all)

# Drop rows where features aren't available due to lag/rolling windows
mask = X_all.notna().all(axis=1)
X_all = X_all[mask]
y_all = y_all[mask]

# 3) Train on 2020-2023
train_end = pd.Timestamp("2023-12-01")  # monthly start index
X_train = X_all.loc[:train_end].copy()
y_train = y_all.loc[:train_end].copy()

if len(X_train) == 0:
    raise ValueError("No training rows found. Check month parsing and file contents.")

model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=0)),
])

model.fit(X_train, y_train)

# 4) Recursive backtest forecast for 2024 (month-by-month)
forecast_months = pd.date_range("2024-01-01", "2024-12-01", freq="MS")

# history contains only data up to 2023-12 for a true forecast-style backtest
history = df["price"].loc[:train_end].copy()

preds = []
for m in forecast_months:
    # Extend series to include month m
    temp = history.copy()
    if m not in temp.index:
        temp.loc[m] = np.nan
    temp = temp.sort_index()

    # Build features and select row for month m
    X_temp = make_features(temp)
    x_m = X_temp.loc[[m]].copy()

    if x_m.isna().any(axis=1).iloc[0]:
        missing_cols = x_m.columns[x_m.isna().iloc[0]].tolist()
        raise ValueError(f"Not enough history to compute features for {m.date()}. Missing: {missing_cols}")

    yhat = float(model.predict(x_m)[0])
    preds.append((m, yhat))

    # Recursive update: predicted value becomes available for next months' lags/rollings
    history.loc[m] = yhat

pred_df = pd.DataFrame(preds, columns=["month", "price"])

# REQUIRED output month format: YYYY-MM-01
pred_df["month"] = pred_df["month"].dt.strftime("%Y-%m-01")

# 5) Save output
pred_df.to_csv("strawberry-backtest.tsv", sep="\t", index=False)

print("Saved: strawberry-backtest.tsv")
print(pred_df.head())


Saved: strawberry-backtest.tsv
        month     price
0  2024-01-01  4.719342
1  2024-02-01  4.566903
2  2024-03-01  4.236359
3  2024-04-01  4.415973
4  2024-05-01  4.082821


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [39]:
# YOUR CHANGES HERE

import pandas as pd

# Load ground truth 
truth = pd.read_csv("strawberry-prices.tsv", sep="\t")
truth.columns = [c.strip().lower() for c in truth.columns]
truth["month"] = pd.to_datetime(truth["month"])
truth = truth.sort_values("month")

truth_2024 = truth[(truth["month"] >= "2024-01-01") & (truth["month"] <= "2024-12-31")].copy()
truth_2024["month"] = truth_2024["month"].dt.to_period("M").dt.to_timestamp()  # month start

# Load predictions
pred = pd.read_csv("strawberry-backtest.tsv", sep="\t")
pred.columns = [c.strip().lower() for c in pred.columns]
pred["month"] = pd.to_datetime(pred["month"])
pred = pred.sort_values("month")
pred["month"] = pred["month"].dt.to_period("M").dt.to_timestamp()  # month start

# Merge
merged = pd.merge(
    truth_2024[["month", "price"]].rename(columns={"price": "actual"}),
    pred[["month", "price"]].rename(columns={"price": "pred"}),
    on="month",
    how="inner",
)

if len(merged) == 0:
    raise ValueError("No overlapping months found between predictions and 2024 ground truth.")

# residual = actual - predicted
merged["residual"] = merged["actual"] - merged["pred"]

mean_resid = float(merged["residual"].mean())
std_resid = float(merged["residual"].std(ddof=1))  # sample std

out = pd.DataFrame({"mean": [mean_resid], "std": [std_resid]})
out.to_csv("backtest-accuracy.tsv", sep="\t", index=False)

print("Saved: backtest-accuracy.tsv")
print(out)


Saved: backtest-accuracy.tsv
       mean       std
0 -0.476187  0.320536


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [40]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

# Load data
df = pd.read_csv("strawberry-prices.tsv", sep="\t")
df.columns = [c.strip().lower() for c in df.columns]

if "month" not in df.columns or "price" not in df.columns:
    raise ValueError("Expected columns: 'month' and 'price' in strawberry-prices.tsv")

df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

# Keep 2020-2024 only
df = df[(df["month"] >= "2020-01-01") & (df["month"] <= "2024-12-31")].copy()

# Ensure monthly start index
df = df.set_index("month").asfreq("MS")

# Fill any missing prices so lags/rollings work
if df["price"].isna().any():
    df["price"] = df["price"].interpolate(limit_direction="both")

# Feature engineering (same as Part 1)
def make_features(series: pd.Series) -> pd.DataFrame:
    X = pd.DataFrame(index=series.index)

    # Trend
    X["t"] = np.arange(len(series))

    # Month-of-year dummies (seasonality)
    month_dummies = pd.get_dummies(series.index.month, prefix="m", drop_first=False)
    month_dummies.index = series.index
    X = pd.concat([X, month_dummies], axis=1)

    # Lags
    X["lag1"] = series.shift(1)
    X["lag12"] = series.shift(12)

    # Rolling means (use only past information)
    X["roll3"] = series.shift(1).rolling(3).mean()
    X["roll6"] = series.shift(1).rolling(6).mean()

    return X

# Build features for all available data
y_all = df["price"].copy()
X_all = make_features(y_all)

# Drop rows where lag/rolling features are not available
mask = X_all.notna().all(axis=1)
X_train = X_all[mask].copy()
y_train = y_all[mask].copy()

if len(X_train) == 0:
    raise ValueError("No training rows found. Check your dates and file contents.")

# Train model on 2020-2024
model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=0)),
])
model.fit(X_train, y_train)

#  Recursive forecast for 2025
forecast_months = pd.date_range("2025-01-01", "2025-12-01", freq="MS")

# history includes ALL known actuals through 2024-12
history = df["price"].copy()

preds_2025 = []
for m in forecast_months:
    temp = history.copy()
    if m not in temp.index:
        temp.loc[m] = np.nan
    temp = temp.sort_index()

    X_temp = make_features(temp)
    x_m = X_temp.loc[[m]].copy()

    if x_m.isna().any(axis=1).iloc[0]:
        missing_cols = x_m.columns[x_m.isna().iloc[0]].tolist()
        raise ValueError(f"Not enough history to compute features for {m.date()}. Missing: {missing_cols}")

    yhat = float(model.predict(x_m)[0])
    preds_2025.append((m, yhat))

    # Recursive update
    history.loc[m] = yhat

forecast_df = pd.DataFrame(preds_2025, columns=["month", "price"])
forecast_df["month"] = forecast_df["month"].dt.strftime("%Y-%m-01")  # REQUIRED FORMAT

forecast_df.to_csv("strawberry-forecast.tsv", sep="\t", index=False)

print("Saved: strawberry-forecast.tsv")
print(forecast_df.head())


Saved: strawberry-forecast.tsv
        month     price
0  2025-01-01  4.663157
1  2025-02-01  4.269221
2  2025-03-01  3.780427
3  2025-04-01  3.850567
4  2025-05-01  3.526983


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [41]:
# YOUR CHANGES HERE

import math
import pandas as pd

BUDGET = 1_000_000.0
FREEZE_COST = 0.20
STORAGE_PER_MONTH = 0.10
SELL_MULT = 0.90  # 10% discount => sell revenue = 0.9 * sell_price

# Load 2025 forecast
fc = pd.read_csv("strawberry-forecast.tsv", sep="\t")
fc.columns = [c.strip().lower() for c in fc.columns]

if "month" not in fc.columns or "price" not in fc.columns:
    raise ValueError("Expected columns: 'month' and 'price' in strawberry-forecast.tsv")

fc["month"] = pd.to_datetime(fc["month"]).dt.to_period("M").dt.to_timestamp()
fc = fc.sort_values("month").reset_index(drop=True)

months = fc["month"].tolist()
prices = fc["price"].astype(float).tolist()

def month_diff(buy_m: pd.Timestamp, sell_m: pd.Timestamp) -> int:
    return (sell_m.year - buy_m.year) * 12 + (sell_m.month - buy_m.month)

rows = []

for i in range(len(months)):
    buy_month = months[i]
    buy_price = float(prices[i])

    for j in range(i + 1, len(months)):
        sell_month = months[j]
        sell_price = float(prices[j])

        m = month_diff(buy_month, sell_month)  # months stored

        cost_per_pint = buy_price + FREEZE_COST + STORAGE_PER_MONTH * m
        revenue_per_pint = SELL_MULT * sell_price
        profit_per_pint = revenue_per_pint - cost_per_pint

        # "buy as many pints as possible" given budget covers all costs
        if cost_per_pint > 0:
            pints = int(math.floor(BUDGET / cost_per_pint))
        else:
            pints = 0

        # If profit per pint is negative, total expected profit will be negative.
        # (Autograder usually still expects the row; do NOT drop it.)
        expected_profit = pints * profit_per_pint

        rows.append({
            "buy_month": buy_month,
            "sell_month": sell_month,
            "pints_purchased": pints,
            "expected_profit": float(expected_profit),
        })

out = pd.DataFrame(rows)

# Required month formatting
out["buy_month"] = pd.to_datetime(out["buy_month"]).dt.strftime("%Y-%m-01")
out["sell_month"] = pd.to_datetime(out["sell_month"]).dt.strftime("%Y-%m-01")

# Column order exactly as requested
out = out[["buy_month", "sell_month", "pints_purchased", "expected_profit"]]

out.to_csv("timings.tsv", sep="\t", index=False)

print("Saved: timings.tsv")
print("Rows:", len(out))   # should be 66 for 12 months
print(out.head())


Saved: timings.tsv
Rows: 66
    buy_month  sell_month  pints_purchased  expected_profit
0  2025-01-01  2025-02-01           201484   -225834.929950
1  2025-01-01  2025-03-01           197505   -328010.947117
2  2025-01-01  2025-04-01           193679   -328798.611573
3  2025-01-01  2025-05-01           190000   -396885.712936
4  2025-01-01  2025-06-01           186457   -452894.551459


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [42]:
# YOUR CHANGES HERE

import pandas as pd

# Load timings (all pairs)
timings = pd.read_csv("timings.tsv", sep="\t")
timings.columns = [c.strip().lower() for c in timings.columns]

needed = {"buy_month", "sell_month", "pints_purchased", "expected_profit"}
if not needed.issubset(timings.columns):
    raise ValueError("timings.tsv must contain buy_month, sell_month, pints_purchased, expected_profit")

# pick best scenario (max expected_profit)
best_row = timings.loc[timings["expected_profit"].astype(float).idxmax()]

best_profit = float(best_row["expected_profit"])
pints = float(best_row["pints_purchased"])

# Load backtest accuracy
acc = pd.read_csv("backtest-accuracy.tsv", sep="\t")
acc.columns = [c.strip().lower() for c in acc.columns]

if "std" not in acc.columns:
    raise ValueError("backtest-accuracy.tsv must contain column 'std'")

std_resid = float(acc.loc[0, "std"])

# one_std_profit = pints_bought_in_best_scenario * std(residual)
one_std_profit = pints * std_resid

out = pd.DataFrame({
    "best_profit": [best_profit],
    "one_std_profit": [one_std_profit],
})

out.to_csv("check.tsv", sep="\t", index=False)

print("Saved: check.tsv")
print(out)


Saved: check.tsv
     best_profit  one_std_profit
0  104757.909647    81854.691602


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


In [43]:
content = """Acknowledgments


I used the Python standard scientific computing stack for this project, including:
- NumPy: for numerical computation, linear algebra, simulation, and resampling.
- pandas: for tabular data handling and saving TSV output files.


I did not discuss this assignment with other students or individuals.


Generative AI:
I used OpenAI’s ChatGPT to help with some coding error to better understanding what was wrong with my codes.


No other outside sources or libraries were used.
"""


with open("acknowledgments.txt", "w") as f: f.write(content)


print("Saved acknowledgments.txt")

Saved acknowledgments.txt


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.